In [2]:
# Attempt at blob detection in 3d images
import skimage as sk
import numpy as np
from skimage import io
from skimage import feature
import pandas as pd
from scipy.spatial import distance as dist
import scipy.cluster.hierarchy as hier
import matplotlib.pyplot as plt
%matplotlib inline

img_file = '/Users/jwilmot/Desktop/SampleGFP.tif'
img_raw = sk.io.imread(img_file)

In [43]:
# 3D Blob Detection Functions
# takes image, min_sigma (affects minimum cell size found), max_sigma (affects max cell size found),
# threshold (affects how bright cells must be to be detected)

# These do NOT perform blob detection in 3D. Rather, they perform 2D blob detection across all planes of the image
# and return an array of y, x, size, z

# Laplacian of Gaussian
def detect_blobs_3dlog(image, min_sigma=10, max_sigma=20, threshold=.003):
    for i in range(len(image)):
        # the if statement is necessary to create the proper array on the first iteration
        if i == 0:
            log_cells = sk.feature.blob_log(image[0,:,:], min_sigma = min_sigma, max_sigma = max_sigma, 
                                            threshold = threshold)
            log_cells = np.column_stack((log_cells, np.ones((len(log_cells)))))
        else:
            new_cells = sk.feature.blob_log(image[i,:,:], min_sigma = min_sigma, max_sigma = max_sigma, 
                                            threshold = threshold)
            new_cells = np.column_stack((new_cells, np.full((len(new_cells)), i+1)))
            log_cells = np.concatenate((log_cells, new_cells))
    return log_cells

# Determinant of Hessian
def detect_blobs_3dhess(image, min_sigma=10, max_sigma=20, threshold=.00008):
    for i in range(len(image)):
        # the if statement is necessary to create the proper array on the first iteration
        if i == 0:
            hess_cells = sk.feature.blob_doh(image[0,:,:], min_sigma = min_sigma,
                                                        max_sigma = max_sigma, threshold = threshold)
            hess_cells = np.column_stack((hess_cells, np.ones((len(hess_cells)))))
        else:
            new_cells = sk.feature.blob_doh(image[i,:,:], min_sigma = min_sigma,
                                                        max_sigma = max_sigma, threshold = threshold)
            new_cells = np.column_stack((new_cells, np.full((len(new_cells)), i+1)))
            hess_cells = np.concatenate((hess_cells, new_cells))
    return hess_cells

# Difference of Gaussian
def detect_blobs_3ddog(image, min_sigma=10, max_sigma=20, threshold=.003):
    for i in range(len(image)):
        # the if statement is necessary to create the proper array on the first iteration
        if i == 0:
            dog_cells = sk.feature.blob_dog(image[0,:,:], min_sigma = min_sigma,
                                                        max_sigma = max_sigma, threshold = threshold)
            dog_cells = np.column_stack((dog_cells, np.ones((len(dog_cells)))))
        else:
            new_cells = sk.feature.blob_dog(image[i,:,:], min_sigma = min_sigma,
                                                        max_sigma = max_sigma, threshold = threshold)
            new_cells = np.column_stack((new_cells, np.full((len(new_cells)), i+1)))
            dog_cells = np.concatenate((dog_cells, new_cells))
    return dog_cells

In [44]:
# Function for performing unsupervised clustering of centroids to get a single centroid for each cell.
# takes vector of x values, vector of y values, and the maximum distance btwn centroids that form a cluster.

def cluster_centroids(x, y, max_dist):
    xy_array = np.column_stack((x,y))
    coordinates = pd.DataFrame(xy_array, columns = ['x','y'])
    
    #create matrix of distances between all centroids
    dist_mat = dist.squareform(dist.pdist(coordinates.values))
    
    #perform clustering
    link_mat = hier.linkage(dist_mat)
    cluster_idx = hier.fcluster(link_mat, max_dist, criterion = 'distance')
    
    #make organized dataframe with labeled clusters
    coordinates['new_label'] = cluster_idx
    coordinates.set_index('new_label', drop=True, append=False, inplace=True)
    coordinates.index.name = 'label'
    coordinates = coordinates.sort_index()
    
    #compute a final centroid for each cell by averaging the x y coords of each cluster
    coordinates = coordinates.groupby(level ='label').mean()
    return coordinates

In [45]:
def plot_blobs(blobs, color, size_mod, fill = True):
    for blob in blobs:
        y, x, size = blob
        c = plt.Circle((x, y), size*size_mod, color=color, linewidth=2, fill=fill)
        ax.add_patch(c)
        

In [52]:
# Example pipeline

# 3 separate blob detections
log_cells = detect_blobs_3dlog(img_raw)
dog_cells = detect_blobs_3ddog(img_raw)
hess_cells = detect_blobs_3dhess(img_raw)

# place into one array
all_cells = np.concatenate((log_cells, dog_cells, hess_cells))

# cluster centroids
final_cells = cluster_centroids(all_cells[:,1], all_cells[:,0], 100)

# Plotting
# Plot on top of max intensity z projection
img_max = img_raw.max(axis=0)
f, ax = plt.subplots(1, figsize = (20,20))
plt.imshow(img_max)
plt.scatter(final_cells['x'], final_cells['y'], 200,  'r')

/Users/jwilmot/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  if sys.path[0] == '':


,x,y
label,,
1,150.388889,408.861111
2,204.567568,437.432432
3,440.419355,57.580645
4,570.310345,152.827586
5,524.428571,219.333333
6,335.368421,104.526316
7,345.000000,85.000000
8,343.000000,88.000000
9,113.000000,173.000000
